### Examples Of Pyspark ML

In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Missing').getOrCreate()

In [3]:
## Read The dataset
training = spark.read.csv(r'test1.csv',header=True,inferSchema=True)

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/C:/Users/hi/pyspark/test1.csv.

In [73]:
training.show()



+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [74]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [75]:
training.columns

['Name', 'age', 'Experience', 'Salary']

In [ ]:
[Age,Experience]----> new feature--->independent feature

In [76]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["age","Experience"],outputCol="Independent Features")

In [77]:
output=featureassembler.transform(training)

In [78]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [79]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Features']

In [80]:
finalized_data=output.select("Independent Features","Salary")

In [81]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [82]:
from pyspark.ml.regression import LinearRegression
##train test split
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor=regressor.fit(train_data)

In [83]:
### Coefficients
regressor.coefficients

DenseVector([-5000.0, 7000.0])

In [84]:
### Intercepts
regressor.intercept

118999.99999893687

In [85]:
### Prediction
pred_results=regressor.evaluate(test_data)

In [86]:
pred_results.predictions.show()

+--------------------+------+-----------------+
|Independent Features|Salary|       prediction|
+--------------------+------+-----------------+
|          [21.0,1.0]| 15000|20999.99999996154|
|          [29.0,4.0]| 20000|2000.000000192551|
|         [31.0,10.0]| 30000|33999.99999993094|
+--------------------+------+-----------------+



In [87]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

(9333.333333233308, 125333333.3306847)